In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
road = cv2.imread('../DATA/road_image.jpg')

In [12]:
road_copy = road.copy()

In [13]:
# plt.imshow(road_copy)

In [14]:
road.shape[:2]

(600, 800)

In [15]:
marker_image = np.zeros(road.shape[:2],dtype=np.int32)

In [16]:
segments = np.zeros(road.shape,dtype=np.uint8)

In [18]:
marker_image.shape

(600, 800)

In [19]:
segments.shape

(600, 800, 3)

In [20]:
# https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
# wybrano qualitative colormaps
from matplotlib import cm

In [21]:
cm.tab10(0) #RGB + A

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)

In [22]:
def create_rgb(i):
    return tuple(np.array(cm.tab10(i)[:3])*255)

In [23]:
colors = []
for i in range(10):
    colors.append(create_rgb(i))

In [24]:
colors

[(31.0, 119.0, 180.0),
 (255.0, 127.0, 14.0),
 (44.0, 160.0, 44.0),
 (214.0, 39.0, 40.0),
 (148.0, 103.0, 189.0),
 (140.0, 86.0, 75.0),
 (227.0, 119.0, 194.0),
 (127.0, 127.0, 127.0),
 (188.0, 189.0, 34.0),
 (23.0, 190.0, 207.0)]

In [25]:
# globalne zmienne:

n_markers = 10 # 0-9

# wybrany kolor
current_marker = 1

# aktualizacja markera przez watershed
marks_updated = False

In [26]:
# callback functions
def mouse_callback(event,x,y,flags,param):
    global marks_updated
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # markery przesłane do algorytmu watershed
        cv2.circle(marker_image,(x,y),10,(current_marker),-1)
        
        # co widzi użytkownik na obrazku
        cv2.circle(road_copy,(x,y),10,colors[current_marker],-1)
        
        marks_update = True

In [28]:
# while true
cv2.namedWindow('Road Image')
cv2.setMouseCallback('Road Image',mouse_callback)

while True:
    cv2.imshow('Watershed Segments', segments)
    cv2.imshow('Road Image', road_copy)
    
    # zamykanie okien
    k = cv2.waitKey(1)
    
    if k == 27:
        break
    
    # czyszczenie kolorów - klawisz "c"
    elif k == ord('c'):
        road_copy = road.copy()
        marker_image = np.zeros(road.shape[0:2],dtype=np.int32)
        segments = np.zeros(road.shape,dtype=np.uint8)
    
    # aktualizacja wyboru koloru
    elif k > 0 and chr(k).isdigit():
        current_marker = int(chr(k))
        
    # aktualizacja markera
    if marks_updated:
        
        marker_img_copy = marker_image.copy()
        cv2.watershed(road,marker_img_copy)
        
        segments = np.zeros(road.shape,dtype=np.uint8)
        
        for color_ind in range(n_markers):
            # kolorowane segmentów
            segments[marker_img_copy==(color_ind)] = colors[color_ind]
            
        marks_updated = False
    
cv2.destroyAllWindows()